## Reconstructing images from few pixels

In [ ]:
from pit import *
import scipy.ndimage as spimg
from scipy.misc import imsave
import matplotlib.pyplot as plt
import glob
import os

def rgb2gray(rgb):
    '''Convert from rgb to grayscale'''
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

### ---------- set image folder -------------------
folder = "/Users/Martin/Pictures/Camera Roll/"
### -----------------------------------------------


# all jpg files in folder
list_camera_pics = glob.glob(folder+'*.jpg') 
# file name of latest one
list_camera_pics=sorted(list_camera_pics, key=os.path.getctime) 
pic = list_camera_pics[-2] 

#read image
# Xorig = spimg.imread(latest_pic,flatten=True, mode='L')
Xorig = rgb2gray(plt.imread(pic))

# # decrease resolution to increase speed
# relative_resolution = .5
# Xorig = spimg.zoom(Xorig, relative_resolution)

# picture size
shape = Xorig.shape
n = np.prod(shape)

# choose frame
wlet = 'db12'
# wlet = 'coif12'
# wlet = 'sym12'
# print(pywt.wavelist(kind='discrete'))
L = 3 
# amplification of lower level wavelet coefficients
# (shows slightly better reconstruction performance)
amp = np.linspace(1,.5,L)
amp = np.kron(amp, np.ones(3) )
amp = np.insert(amp,0, 12 ) #prepend (position 0)

### A compression test

In [ ]:
## (not necessary for the rest)
# choose basis for compression
T = WT(shape, wavelet=wlet,level=L, amplify=amp)

# hard TO
p = 0.03
TO = hardTO( p*np.prod(shape) )

# compression test
cXorig = compress(T,TO,Xorig)

print("Original image:")
pltPic(Xorig)

print("Compressed image (down to {:0.1f}% coefficients of {}):".format(100*p,n) )
pltPic(cXorig)

### Remove pixels

In [ ]:
# set relative number of pixels
p = .05

# total number of pixels
m = int(p*n)
print("Total number of pixels: {}k".format(int(np.prod(Xorig.shape)/1000)))
print("Number of remaining pixels: {}k".format( int(m/1000) ))

# generate subsampled picture
mask = getRandMask(n, m)
Xsub = np.zeros(shape)
Xsub.flat[mask] = Xorig.flat[mask]
pltPic(Xsub)

In [ ]:
# schedule for DCT and WT iterations
steps_wt = 14
steps_dct = 10 #parameter for nr. of steps in fixed basis
N = 5 #nr. of switching bases: wt and dct

# threshold values
th_dct = 8
th_wt = 12

# set initial guess for the reconstruction
if 'Xrec' not in locals():
    Xrec = Xsub
    
Xrec = Xsub

# set transformations
dct = DCT(shape)
wt = WT(shape, wavelet=wlet,level=L, amplify=amp)

for j in range(N):
    thOp = softTO(th_dct)
    Xrec=FISTA(dct, thOp, mask, Xsub, stepsize = .75, n_steps=steps_dct, Xorig=Xorig, X0=Xrec)
    
    thOp = softTO(th_wt)
    Xrec=FISTA(wt, thOp, mask, Xsub, stepsize = .75, n_steps=steps_wt, Xorig=Xorig, X0=Xrec)

pltPic(Xrec)

In [ ]:
pltPic(Xorig)